<a href="https://colab.research.google.com/github/cincysam6/Field_Control_Model/blob/main/interactive_field_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# --- CONFIG: point these at your repo ---
REPO_URL = "https://github.com/cincysam6/Field_Control_Model.git"
REPO_DIR = "/content/Field_Control_Model"      # where to clone
PKG_ROOT = REPO_DIR                             # path to add to sys.path
REMOTE = "origin"
BRANCH = "main"                                 # or 'master' if you use that

# If your repo is private, set a token (or leave as None for public)
GITHUB_TOKEN = None  # e.g., "ghp_xxx..." or None for public

# ------------- helper: clone or pull -------------
import os, sys, subprocess, textwrap

def _run(cmd, cwd=None, check=True):
    print(" ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=check, text=True, capture_output=True)

def ensure_repo(repo_url: str, repo_dir: str, remote="origin", branch="main", token=None, hard_reset=False):
    # Support private repos via token
    if token and repo_url.startswith("https://"):
        repo_url = repo_url.replace("https://", f"https://{token}@")

    if not os.path.exists(repo_dir):
        print(f"Cloning into {repo_dir} ...")
        _run(["git", "clone", "--recursive", repo_url, repo_dir])
    else:
        print(f"Repo exists at {repo_dir}. Fetching updates ...")
        _run(["git", "fetch", remote], cwd=repo_dir)
        if hard_reset:
            print(f"Hard resetting to {remote}/{branch} ...")
            _run(["git", "reset", "--hard", f"{remote}/{branch}"], cwd=repo_dir)
        else:
            print("Pulling latest changes ...")
            # Fast-forward only; avoids merge prompts in Colab
            try:
                _run(["git", "pull", "--ff-only", remote, branch], cwd=repo_dir)
            except subprocess.CalledProcessError as e:
                print("Fast-forward failed (local changes?). Falling back to hard reset.")
                _run(["git", "reset", "--hard", f"{remote}/{branch}"], cwd=repo_dir)

    # Ensure on the right branch
    _run(["git", "checkout", branch], cwd=repo_dir)

# ------------- do the fetch/pull -------------
ensure_repo(REPO_URL, REPO_DIR, remote=REMOTE, branch=BRANCH, token=GITHUB_TOKEN, hard_reset=False)

# ------------- put repo on sys.path -------------
if PKG_ROOT not in sys.path:
    sys.path.append(PKG_ROOT)

# ------------- turn on autoreload -------------
%load_ext autoreload
%autoreload 2

print("✅ Repo ready, path set, autoreload ON.")


Repo exists at /content/Field_Control_Model. Fetching updates ...
git fetch origin
Pulling latest changes ...
git pull --ff-only origin main
git checkout main
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ Repo ready, path set, autoreload ON.


In [12]:
# 1. Clone your repo
#!git clone https://github.com/cincysam6/Field_Control_Model.git

# 2. Add it to Python path
import sys
#sys.path.append('/content/Field_Control_Model')

# 3. Import your helper function
from src.helpers import compute_distances_by_frame, pick_panel_kwargs,update_kwargs
from src.diagnostic_plots import plot_dir_orientation_small_multiples,diagnostic_plot,plot_field_control_small_multiples,diagnostic_multiples
from src.field_control_model import PlayerInfluenceModel
from src.compute_player_density import compute_player_densities_dataframe
from src.plot_player_density import plot_player_densities_from_dataframe, plot_team_densities_small_multiples
from src.animate_plays import animate_pitch_control_with_players, animate_pitch_control_with_players_fast
from src.compute_player_density import compute_player_densities_team_control
from src.plot_player_density import visualize_team_control
from src.presets import default_kwargs, triangular_kwargs

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrow
from matplotlib.lines import Line2D
import ipywidgets as w
from ipywidgets import Layout, VBox, HBox, Accordion, Tab, Output


In [16]:


# ---------- Presets (edit to taste) ----------
PRESETS = {
    "Balanced": dict(
        gaussian_scale_factor=0.68,
        alpha_gamma=6.0,
        beta_min=2.0, beta_max=18.0,
        gamma_midpoint=13.0,
        gamma_scale_factor=0.9,
        max_forward_distance=20.0,
        forward_decay_factor=1.1,
        angle_limit_min=15.0, angle_limit_max=55.0, angle_decay_factor=2.2,
        w_gaussian_min=0.2, w_gaussian_max=1.0,
        gaussian_midpoint=4.0, gaussian_steepness=1.8,
    ),
    "Conservative": dict(
        gaussian_scale_factor=0.7,
        alpha_gamma=11.0,
        beta_min=2.0, beta_max=15.0,
        gamma_midpoint=15.0,
        gamma_scale_factor=0.8,
        max_forward_distance=18.0,
        forward_decay_factor=1.0,
        angle_limit_min=20.0, angle_limit_max=50.0, angle_decay_factor=2.5,
        w_gaussian_min=0.3, w_gaussian_max=1.0,
        gaussian_midpoint=4.5, gaussian_steepness=2.0,
    ),
    "Aggressive wedge": dict(
        gaussian_scale_factor=0.65,
        alpha_gamma=3.5,
        beta_min=1.0, beta_max=25.0,
        gamma_midpoint=10.0,
        gamma_scale_factor=1.0,
        max_forward_distance=25.0,
        forward_decay_factor=1.3,
        angle_limit_min=12.0, angle_limit_max=60.0, angle_decay_factor=2.0,
        w_gaussian_min=0.15, w_gaussian_max=1.0,
        gaussian_midpoint=3.5, gaussian_steepness=1.5,
    ),
}

# ---------- Controls ----------
# Scene
speed      = w.FloatSlider(5.0,  min=0.0, max=11.5, step=0.1, description="Speed")
dir_deg    = w.FloatSlider(0.0,  min=0.0, max=360.0, step=1.0, description="Direction°")
show_o     = w.Checkbox(False, description="Show Orientation")
o_deg      = w.FloatSlider(0.0,  min=0.0, max=360.0, step=1.0, description="Orient°")
x          = w.FloatSlider(60.0, min=0.0, max=120.0, step=0.5, description="X")
y          = w.FloatSlider(26.65,min=0.0, max=53.3, step=0.5, description="Y")
zoom       = w.Checkbox(False, description="Zoom around X")
zoom_pad_x = w.FloatSlider(10.0, min=2.0, max=30.0, step=1.0, description="Zoom pad")
name       = w.Text(value="Adam Thielen", description="Name")

# Gaussian
gaussian_scale_factor = w.FloatSlider(0.68, min=0.4, max=1.2, step=0.02, description="Scale")

# Gamma
alpha_gamma       = w.FloatSlider(6.0,  min=2.0,  max=12.0, step=0.5, description="alpha")
beta_min          = w.FloatSlider(2.0,  min=0.5,  max=8.0,  step=0.5, description="beta_min")
beta_max          = w.FloatSlider(18.0, min=8.0,  max=30.0, step=1.0, description="beta_max")
gamma_midpoint    = w.FloatSlider(13.0, min=6.0,  max=20.0, step=0.5, description="midpoint")
gamma_scale_factor= w.FloatSlider(0.9,  min=0.6,  max=1.4, step=0.02, description="scale_fac")
max_forward_distance = w.FloatSlider(20.0, min=10.0, max=30.0, step=1.0, description="cap")
forward_decay_factor = w.FloatSlider(1.1, min=0.6, max=2.0, step=0.05, description="tail_decay")

# Cone
angle_limit_min   = w.FloatSlider(15.0, min=8.0,  max=30.0, step=1.0, description="min° (fast)")
angle_limit_max   = w.FloatSlider(55.0, min=35.0, max=80.0, step=1.0, description="max° (slow)")
angle_decay_factor= w.FloatSlider(2.2,  min=1.0,  max=4.0,  step=0.1, description="decay")

# Weights
w_gaussian_min    = w.FloatSlider(0.2, min=0.0, max=0.5, step=0.02, description="wG min")
w_gaussian_max    = w.FloatSlider(1.0, min=0.5, max=1.0, step=0.02, description="wG max")
gaussian_midpoint = w.FloatSlider(4.0, min=2.0, max=6.0, step=0.1, description="midpoint")
gaussian_steepness= w.FloatSlider(1.8, min=0.5, max=3.0, step=0.1, description="steepness")

# Style
levels     = w.IntSlider(28, min=10, max=60, step=1, description="Levels")
cmap       = w.Dropdown(options=["OrRd","Reds","magma","inferno"], value="OrRd", description="Colormap")
fill_alpha = w.FloatSlider(0.22, min=0.05, max=0.5, step=0.01, description="Fill α")
line_alpha = w.FloatSlider(0.50, min=0.1,  max=0.9, step=0.01, description="Line α")
line_lw    = w.FloatSlider(0.6,  min=0.2,  max=2.0, step=0.1,  description="Line lw")

# Preset & buttons
preset_dd  = w.Dropdown(options=list(PRESETS.keys()), value="Balanced", description="Preset")
apply_btn  = w.Button(description="Apply Preset", button_style="info")
reset_btn  = w.Button(description="Reset Sliders", button_style="warning")

# ---------- Layout: left panel (Accordion) ----------
scene_box = VBox([name, speed, dir_deg, show_o, o_deg, x, y, zoom, zoom_pad_x])
gauss_box = VBox([gaussian_scale_factor])
gamma_box = VBox([alpha_gamma, beta_min, beta_max, gamma_midpoint, gamma_scale_factor,
                  max_forward_distance, forward_decay_factor])
cone_box  = VBox([angle_limit_min, angle_limit_max, angle_decay_factor])
weight_box= VBox([w_gaussian_min, w_gaussian_max, gaussian_midpoint, gaussian_steepness])
style_box = VBox([levels, cmap, fill_alpha, line_alpha, line_lw])

acc = Accordion(children=[scene_box, gauss_box, gamma_box, cone_box, weight_box, style_box])
for i, title in enumerate(["Scene", "Gaussian", "Gamma", "Cone", "Weights", "Style"]):
    acc.set_title(i, title)

left_panel = VBox([
    HBox([preset_dd, apply_btn, reset_btn]),
    acc
], layout=Layout(width="360px"))

# ---------- Plot output on the right ----------
out = Output(layout=Layout(border="1px solid #eee",
                           width="100%",
                           height="700px"))

# ---------- helpers ----------
def _add_arrow(ax, start, theta_deg, length=2.2, body=0.08, head_w=0.35, head_l=0.70, color="#2F6CFF", z=6):
    x0, y0 = start
    th = np.deg2rad(90 - theta_deg)
    dx, dy = length*np.cos(th), length*np.sin(th)
    ax.add_patch(FancyArrow(
        x0, y0, dx, dy,
        width=body, head_width=head_w, head_length=head_l,
        length_includes_head=True, color=color, alpha=0.9, linewidth=0.6, zorder=z
    ))

def _draw_field(ax, x_max=120.0, y_max=53.3, zoom=False, x0=None, pad=10.0):
    ax.set_xlim(0, x_max); ax.set_ylim(0, y_max)
    if zoom and x0 is not None:
        ax.set_xlim(x0-pad, x0+pad)
    ax.set_aspect('equal', adjustable='box')
    ax.set_facecolor("#FAFBFD")
    for xv in np.arange(0, x_max+1, 5):
        ax.axvline(xv, color="#D7DEE8", lw=0.6, alpha=0.6, zorder=0)
    for yv in np.arange(0, y_max+1, 5):
        ax.axhline(yv, color="#EFF3F8", lw=0.6, alpha=0.6, zorder=0)
    for sp in ax.spines.values():
        sp.set_visible(False)
    ax.tick_params(colors="#8A94A6", labelsize=9)
    ax.set_xlabel("X (yards)", color="#444B59")
    ax.set_ylabel("Y (yards)", color="#444B59")

def _current_model_kwargs():
    return dict(
        gaussian_scale_factor=gaussian_scale_factor.value,
        alpha_gamma=alpha_gamma.value,
        beta_min=beta_min.value, beta_max=beta_max.value,
        gamma_midpoint=gamma_midpoint.value,
        gamma_scale_factor=gamma_scale_factor.value,
        max_forward_distance=max_forward_distance.value,
        forward_decay_factor=forward_decay_factor.value,
        angle_limit_min=angle_limit_min.value,
        angle_limit_max=angle_limit_max.value,
        angle_decay_factor=angle_decay_factor.value,
        w_gaussian_min=w_gaussian_min.value,
        w_gaussian_max=w_gaussian_max.value,
        gaussian_midpoint=gaussian_midpoint.value,
        gaussian_steepness=gaussian_steepness.value,
    )

def _render(*args):
    with out:
        out.clear_output(wait=True)
        model = PlayerInfluenceModel(**_current_model_kwargs())
        pos = (float(x.value), float(y.value))
        o = o_deg.value if show_o.value else None
        pos_off = model.compute_offset(pos, dir_deg.value, speed.value)
        Z = model.base_distribution(
            pos_xy=pos, pos_off_xy=pos_off, direction_deg=dir_deg.value,
            speed=speed.value, dist_from_ball=10.0
        )
        fig, ax = plt.subplots(figsize=(10, 7), dpi=140)
        _draw_field(ax, x_max=model.field_x_max, y_max=model.field_y_max,
                    zoom=zoom.value, x0=pos[0], pad=zoom_pad_x.value)
        extent = [0, model.field_x_max, 0, model.field_y_max]
        ax.contourf(Z, levels=levels.value, cmap=cmap.value, alpha=fill_alpha.value,
                    antialiased=True, origin="lower", extent=extent, zorder=1)
        ax.contour(Z, levels=levels.value, cmap=cmap.value, alpha=line_alpha.value,
                   linewidths=line_lw.value, origin="lower", extent=extent, zorder=2)
        ax.scatter(*pos, s=22, c="#0B57D0", edgecolors="white", linewidths=1.0, zorder=7)
        ax.scatter(*pos, s=8,  c="white", edgecolors="none", zorder=8)
        _add_arrow(ax, pos, dir_deg.value, length=2.2, body=0.08, head_w=0.35, head_l=0.70, color="#2F6CFF", z=9)
        if o is not None:
            _add_arrow(ax, pos, o, length=1.8, body=0.08, head_w=0.32, head_l=0.64, color="#FF8A00", z=8)
        title = f"{name.value} – Speed={speed.value:.2f} yd/s | dir={dir_deg.value:.2f}°"
        if o is not None:
            title += f" | o={o:.2f}°"
        ax.set_title(title, fontsize=16, pad=10, color="#2B3139")
        plt.show()

def _apply_preset(_):
    cfg = PRESETS[preset_dd.value]
    # model knobs
    gaussian_scale_factor.value = cfg["gaussian_scale_factor"]
    alpha_gamma.value           = cfg["alpha_gamma"]
    beta_min.value              = cfg["beta_min"]
    beta_max.value              = cfg["beta_max"]
    gamma_midpoint.value        = cfg["gamma_midpoint"]
    gamma_scale_factor.value    = cfg["gamma_scale_factor"]
    max_forward_distance.value  = cfg["max_forward_distance"]
    forward_decay_factor.value  = cfg["forward_decay_factor"]
    angle_limit_min.value       = cfg["angle_limit_min"]
    angle_limit_max.value       = cfg["angle_limit_max"]
    angle_decay_factor.value    = cfg["angle_decay_factor"]
    w_gaussian_min.value        = cfg["w_gaussian_min"]
    w_gaussian_max.value        = cfg["w_gaussian_max"]
    gaussian_midpoint.value     = cfg["gaussian_midpoint"]
    gaussian_steepness.value    = cfg["gaussian_steepness"]
    _render()

def _reset_sliders(_):
    # reset to "Balanced"
    preset_dd.value = "Balanced"
    _apply_preset(None)

apply_btn.on_click(_apply_preset)
reset_btn.on_click(_reset_sliders)

# Auto-update when any control changes (lightweight debounce via continuous_update)
for ctl in [speed, dir_deg, show_o, o_deg, x, y, zoom, zoom_pad_x,
            gaussian_scale_factor, alpha_gamma, beta_min, beta_max, gamma_midpoint,
            gamma_scale_factor, max_forward_distance, forward_decay_factor,
            angle_limit_min, angle_limit_max, angle_decay_factor,
            w_gaussian_min, w_gaussian_max, gaussian_midpoint, gaussian_steepness,
            levels, cmap, fill_alpha, line_alpha, line_lw]:
    ctl.observe(_render, names="value")

# initial render & display layout
_apply_preset(None)  # loads Balanced and renders once

ui = HBox([left_panel, out], layout=Layout(width="100%"))
ui
